# Imports and settings

In [121]:
from openmm.app import *
from openmm import *              
from openmm.unit import *

# Create system + CV Forces with harmonic restraints

In [122]:
# Load an already solvated PDB file and set up the system + state
pdb = PDBFile("../villin.pdb")
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology, 
                                         nonbondedMethod=PME, 
                                         nonbondedCutoff=10.0 * angstrom, 
                                         constraints=HBonds,
                                         rigidWater=True,
                                         hydrogenMass=4.0 * amu)


In [123]:
# Define three atom indices - these will be used to define distance CVs
d1_atom1_ind = 83
d1_atom2_ind = 151
d2_atom2_ind = 254

In [124]:
# Create Harmonic Force that operate on the CVs 
bias = HarmonicBondForce()
bias.addBond(d1_atom1_ind, 
             d1_atom2_ind, 
             0.3, 
             1.0)

bias.addBond(d1_atom1_ind,
             d2_atom2_ind,
             0.3,
             1.0)

1

In [125]:
# Add force to the system
system.addForce(bias)

5

Ultimately we want to track the value of these distances, which we will use to create our free energy projection – let's call them  D1 and D2.\
To track D1 and D2, we could use a whole `Reporter` object to save the whole trajectory and measure it afterwards.\
A lightweight way of doing this without the whole trajectory is to use a `customCVForce` object set to 0 (i.e. no bias). This will compute the value of a CV at each step of the simulation.\
We then pass D1 and D2 as bondForces to the `customCVForce` object (using `r` to track their distance)

In [126]:
# define a distance measurer
dist_measurer = CustomCVForce("0")
# Create our two distances as separate Bond Forces and add them
# Note: If we added both D1 and D2 to the same BondForce, then doing 
#     dist_measurer.getCollectiveVariableValues will return only the first distance (D1)
# Hence we define them as separate BondForces
D1 = CustomBondForce("r")
D1.addBond(d1_atom2_ind, d1_atom1_ind)
D2 = CustomBondForce("r")
D2.addBond(d2_atom2_ind, d1_atom1_ind)

# Add each BondForce as CVs into the dist_measurer
dist_measurer.addCollectiveVariable("D1", D1)
dist_measurer.addCollectiveVariable("D2", D2)
system.addForce(dist_measurer)

6

# define Integrator and create the Simulation object

In [127]:
integrator = LangevinMiddleIntegrator(300*kelvin,
                                      1.0/picosecond,
                                      0.002 * picosecond)

In [128]:
simulation = Simulation(pdb.topology,system, integrator)

# Set up a single simulation with one umbrella (in reality you'd do multiple of these)


In [129]:
simulation.context.setPositions(pdb.positions)
    

In [131]:
for i in range(5):
    simulation.step(10)
    n_steps = str(simulation.context.getStepCount())
    d1,d2 = dist_measurer.getCollectiveVariableValues(simulation.context)
    print(n_steps, d1, d2)

4050 0.4116576611995697 0.4263102114200592
4060 0.41148799657821655 0.4017527401447296
4070 0.42247557640075684 0.4031124711036682
4080 0.44210001826286316 0.40274685621261597
4090 0.4301850199699402 0.40563341975212097


If we were to do this for multiple windows, the way to do this is set up a loop over the windows, and then run the simulation for each window.\
For each system, you'd just have to first define a set of umbrellas with something like:

```
M = 20
r0_range = np.linspace(0.3, 2.0, M, endpoint = False)
```
Then for each new system (1 per window), you'd add it like: 
```
simulation.context.setParameter('r0_d1', r0_range[m])
```